In [1]:
suppressPackageStartupMessages(library(tidyverse))
suppressPackageStartupMessages(library(data.table))


Warning message:
“package ‘ggplot2’ was built under R version 3.6.2”Warning message:
“package ‘tibble’ was built under R version 3.6.3”Warning message:
“package ‘tidyr’ was built under R version 3.6.2”Warning message:
“package ‘dplyr’ was built under R version 3.6.2”Warning message:
“package ‘forcats’ was built under R version 3.6.2”

In [43]:
elastic_net_phe_f <- '/oak/stanford/groups/mrivas/projects/biobank-methods-dev/snpnet-elastic-net/phenotype.phe'


In [3]:
gwas_covar_f <- '/oak/stanford/groups/mrivas/ukbb24983/sqc/population_stratification_w24983_20190809/ukb24983_GWAS_covar.20190809.phe'


In [6]:
split_d <- '/oak/stanford/groups/mrivas/users/ytanigaw/repos/rivas-lab/biobank-methods-dev/private_data/data-split'


In [4]:
gwas_covar_df <- fread(
    gwas_covar_f,
    select=c('FID', 'IID', 'age', 'sex', paste0('PC', 1:10)),
    colClasses=c('FID'='character', 'IID'='character')
)


In [13]:
split_df <- c('train', 'val', 'test') %>%
lapply(function(s){
    fread(
        file.path(split_d, sprintf('%s.fam', s)), 
        colClasses='character'
    ) %>%
    select(V1, V2) %>%
    rename('FID'='V1', 'IID'='V2') %>%
    mutate(split = s)
}) %>%
bind_rows()


In [22]:
HC269_df <- fread(
    '/oak/stanford/groups/mrivas/ukbb24983/phenotypedata/extras/highconfidenceqc/v1_2017/phe/HC269.phe',
    colClasses=c('character', 'character', 'numeric')
)
colnames(HC269_df) <- c('FID', 'IID', 'HC269')

HC382_df <- fread(
    '/oak/stanford/groups/mrivas/ukbb24983/phenotypedata/extras/highconfidenceqc/v1_2017/phe/HC382.phe',
    colClasses=c('character', 'character', 'numeric')
)
colnames(HC382_df) <- c('FID', 'IID', 'HC382')


In [29]:
INI50_df <- fread(
    '/oak/stanford/groups/mrivas/projects/biobank-methods-dev/snpnet-elastic-net/INI50.phe',
    colClasses=c('character', 'character', 'numeric')
) %>%
mutate(V3 = na_if(V3, -9))
colnames(INI50_df) <- c('FID', 'IID', 'INI50')

INI21001_df <- fread(
    '/oak/stanford/groups/mrivas/projects/biobank-methods-dev/snpnet-elastic-net/INI21001.phe',
    colClasses=c('character', 'character', 'numeric')
) %>%
mutate(V3 = na_if(V3, -9))
colnames(INI21001_df) <- c('FID', 'IID', 'INI21001')



In [40]:
df <- gwas_covar_df %>%
left_join(split_df, by=c('FID', 'IID')) %>%
left_join(HC269_df, by=c('FID', 'IID')) %>%
left_join(HC382_df, by=c('FID', 'IID')) %>%
left_join(INI50_df, by=c('FID', 'IID')) %>%
left_join(INI21001_df, by=c('FID', 'IID'))


In [42]:
df %>% count(split)

split,n
<chr>,<int>
test,67272
train,202276
val,67651
NA,151178


In [46]:
df %>%
drop_na(split) %>%
fwrite(elastic_net_phe_f, sep='\t', na = "NA", quote=F)
